Caso práctico: Extractor de comandas Una importante cadena de restaurantes quiere implantar un sistema de comandas por chat, de esta forma los clientes simplemente tendrán que escribir su comanda por una tablet en la mesa y el asistente virtual se encargará de gestionar el pedido con la cocina y los camareros. El restaurante necesita tener un listado de los platos y las bebidas por separado junto con el número de unidades que se piden de cada elemento.

In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk import UnigramTagger, BigramTagger, TrigramTagger
from nltk.tag.hmm import HiddenMarkovModelTagger

# Regex Parser
from nltk.chunk.regexp import *

# Corpus CESS en español
from nltk.corpus import cess_esp

In [2]:
hmm = HiddenMarkovModelTagger.train(cess_esp.tagged_sents())

def taggear(_frase, _tagger):
    
    tokens = word_tokenize(_frase)
    if _tagger == 'hmm':
        return hmm.tag(tokens)

In [3]:
# Reglas para Comandas
reglas_comandas = '''
    Item: {<nc.*>|<np.*>}
    Tipo: {<nc.*>|<aq.*>|<Fpa>}
    Cantidad: {<Z>}
    Comanda: {<Item><Tipo>?<Cantidad>}
'''

parser_comandas = nltk.RegexpParser(reglas_comandas)

def extrae_comanda(_frase):
    tags = taggear(_frase, 'hmm')
    print("Etiquetas POS:", tags)
    parseo = parser_comandas.parse(tags)
    print("Árbol de análisis:", parseo)
    resultado = []

    for nodo in parseo:
        if isinstance(nodo, nltk.Tree) and nodo.label() == 'Comanda':
            comanda = {'item': '', 'tipo': '', 'cantidad': ''}
            for elm in nodo:
                
                if isinstance(elm, tuple) and len(elm) == 2:
                    palabra, tag = elm
                elif isinstance(elm, nltk.Tree):
                    for sub_elm in elm:
                        if isinstance(sub_elm, tuple) and len(sub_elm) == 2:
                            sub_palabra, sub_tag = sub_elm
                            if elm.label() == 'Item' and ('nc' in sub_tag or 'np' in sub_tag):
                                comanda['item'] = sub_palabra
                            elif elm.label() == 'Tipo' and ('aq' in sub_tag or 'Fpa' in sub_tag):
                                comanda['tipo'] = sub_palabra
                            elif elm.label() == 'Cantidad' and 'Z' in sub_tag:
                                comanda['cantidad'] = sub_palabra
            resultado.append(comanda)

    return resultado


In [4]:
print(extrae_comanda('hamburguesa queso 2'))

Etiquetas POS: [('hamburguesa', 'np0000p'), ('queso', 'Fpa'), ('2', 'Z')]
Árbol de análisis: (S
  (Comanda
    (Item hamburguesa/np0000p)
    (Tipo queso/Fpa)
    (Cantidad 2/Z)))
[{'item': 'hamburguesa', 'tipo': 'queso', 'cantidad': '2'}]


In [5]:
def imprimir_tags(_frase):
    tags = taggear(_frase, 'hmm')
    print(tags)

imprimir_tags('1 hamburguesa queso')


[('1', 'Z'), ('hamburguesa', 'ncmp000'), ('queso', 'aq0mp0')]


No entiendo por qué si cambio la cantidad (1) al principio, el tipo:queso tiene una etiqueta diferente

In [6]:
# Para más de una comanda
reglas_comandas2 = '''
    Item: {<nc.*>|<np.*>}
    Tipo: {<nc.*>|<aq.*>|<Fpa>}
    Cantidad: {<Z>}
    Comanda: {<Item><Tipo>?<Cantidad>}
    Comandas: {<Comanda>(<,><Comanda>)*}
'''
parser_comandas2 = nltk.RegexpParser(reglas_comandas2)

def extrae_comanda2(_frase):
    tags = taggear(_frase, 'hmm')
    parseo = parser_comandas2.parse(tags)
    resultado = []

    for comandas in parseo:
        if isinstance(comandas, nltk.Tree) and comandas.label() == 'Comandas':
            for nodo in comandas:
                if isinstance(nodo, nltk.Tree) and nodo.label() == 'Comanda':
                    comanda = {'item': '', 'tipo': '', 'cantidad': ''}
                    for elm in nodo:
                        if isinstance(elm, tuple) and len(elm) == 2:
                            palabra, tag = elm
                            
                        elif isinstance(elm, nltk.Tree):
                            for sub_elm in elm:
                                if isinstance(sub_elm, tuple) and len(sub_elm) == 2:
                                    sub_palabra, sub_tag = sub_elm
                                    if elm.label() == 'Item' and ('nc' in sub_tag or 'np' in sub_tag):
                                        comanda['item'] = sub_palabra
                                    elif elm.label() == 'Tipo' and ('aq' in sub_tag or 'Fpa' in sub_tag):
                                        comanda['tipo'] = sub_palabra
                                    elif elm.label() == 'Cantidad' and 'Z' in sub_tag:
                                        comanda['cantidad'] = sub_palabra
                    resultado.append(comanda)

    return resultado


In [7]:
extrae_comanda2('hamburguesa queso 3,patatas bravas 1,cocacola zero 2')

[{'item': 'hamburguesa', 'tipo': 'queso', 'cantidad': '3'},
 {'item': 'patatas', 'tipo': 'bravas', 'cantidad': '1'},
 {'item': 'cocacola', 'tipo': 'zero', 'cantidad': '2'}]